## In this session

- How to build a traditional model?
- How to improve model performance?
- What are the drawbacks of 'simple' performance improvement?

### Topics covered
- Linear regression
- RMSE as a measure of fit
- Overfitting

## Linear Regression

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.eval_measures import mse,rmse
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline

In [ ]:
filename = 'used_cars.csv'

In [ ]:
filepath = os.path.join('datasets', filename)
filepath

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
sns.regplot(
    data = df,
    x = 'age', y = 'price',
    marker = '.',
    fit_reg = True, 
    line_kws= {'color': 'k'}
)
plt.title('Price as a linear function of age');

In [ ]:
sns.regplot(
    data = df,
    x = 'age', y = 'price',
    marker = '.',
    fit_reg = True, lowess = True,
    line_kws= {'color': 'k'}
)
plt.title('Price as a non-linear function of age');

In [ ]:
df.groupby('area')['price'].count()

In [ ]:
df_chicago = df[df.chicago == 1]
df_la = df[df.chicago == 0]

In [ ]:
df_la.sort_values(by = 'age', inplace= True)

### Build a Model - A Traditional Approach

In the traditional approach we are defining a set of `explanatory variables`, which we think are related to the development of our `target variable`. The model assumes a `linear relationship` between the target variable $y$ and the explanatory variables. 

<center>
    $y_i = \beta_0 + \beta_1x_i + \epsilon_i$
</center>
<br>where $\beta_0$ is a constant, and $\epsilon$ is a normally distributed random number with an expected value of 0: 
<br>
<br>
<center>
    $\epsilon \sim N(0,\sigma_{\epsilon})$.
</center>
<br>
(Note: standard normal distribution is $N(0, 1)$)

<br>The - almost oversimplifed -  interpretation of the parameters are:
- $\beta_0$: the expected value of $y$ if $x$ = 0 (can be meaningless) 
- $\beta_1$: expected **unit change** in $y$ if $x$ is **one unit higher**, each measured in their own units.

A regression model is meaningful if
- the model explains a fair amount from the variability of the dependent ($y$) variable; we measure it with $R^2$ 
- if the $\beta_1$, $\beta_2$, ... variables are significantly different from zero; we measure it with the _t_statistics_ and the corresponding _p_values_
- the model overall _F-stastistic_ is high and its corresponding _p_value_ is low. 

#### How to build a traditional model in Python?

1. Define your exact model as a formula.
2. Instantiate a regression object.
3. Fit the regression to your data
4. Analyze results
5. Compare your model to other options.
6. Make prediction.

1. **Define your formula**
<center>
    $price_i = \beta_0 + \beta_1age_i + \epsilon_i$
</center>

<br>   

2. **Instantiate a model object**

In [ ]:
simple_model = smf.ols("price ~ age", data=df_chicago)

In [ ]:
type(simple_model)

3. **Fit the regression to your data**

In [ ]:
simple_model_fitted = simple_model.fit(cov_type= 'HC1')

In [ ]:
type(simple_model_fitted)

4. **Analyze results**

In [ ]:
print(simple_model_fitted.get_robustcov_results().summary())

In [ ]:
print('R2 = {:.2%}'.format(simple_model_fitted.get_robustcov_results().rsquared))

In [ ]:
fitted_values_chicago = simple_model_fitted.get_robustcov_results().fittedvalues

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x = df_chicago.age, y = df_chicago.price, marker = '.')
ax.plot( df_chicago.age, fitted_values_chicago, color = 'k', label = 'regression line')
ax.hlines( 0, 0, df_chicago.age.max(), color = 'k', linewidth = 0.5)
plt.title('Original and fitted values as a function of age - Chicago')
plt.legend();

#### External validity

`External validity` is concept related to the behavior of the model on data which it has not seen. If we train the model Chicago, will it properly predict data for cars in Los Angeles?

In [ ]:
fitted_values_la = simple_model_fitted.predict(df_la)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x = df_la.age, y = df_la.price, marker = '.')
ax.plot(df_la.age, fitted_values_la, color = 'k', label = 'regression line')
ax.hlines( 0, 0, df_la.age.max(), color = 'k', linewidth = 0.5)
plt.title('Original and fitted values as a function of age - Los Angeles')
plt.legend();

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (12,6))
axs[0].scatter(x = df_chicago.price, y = fitted_values_chicago, marker = '.', color = 'black')
axs[0].axline([0, 0], [1, 1], color = 'k')
axs[0].set_title('Chicago')
axs[1].scatter(x = df_la.price, y = fitted_values_la, marker = '.', color = 'k')
axs[1].axline([0, 0], [1, 1], color = 'k')
axs[1].set_title('Los Angeles')
fig.suptitle('Original vs predicted values - simple model')

for ax in axs.flat:
    ax.set(xlabel='original', ylabel='fitted/predicted')

for ax in axs.flat:
    ax.label_outer()

#### Measures of fit

We need to compare predicted values with original values. In order to punish large misses in prediction we take the average of the *squared differences* between the original and the predicted data. To make it comparable with the  original values we take square root of this average. 

This measure is of primary importance in modelling tabular data, and we call it `root mean squared error`, or `RMSE`.
$$
RMSE = \sqrt{\frac{ \sum_{i=1}^{n} (y_i - \hat{y_i})^2}{n}}
$$
<br>

where $y_i$ is the original value and $\hat{y_i}$ is the predicted value.

In [ ]:
rmse_chicago = rmse(df_chicago.price, fitted_values_chicago)
rmse_la = rmse(df_la.price, fitted_values_la)

print('RMSE Chicago: {:,.2f}.'.format(rmse_chicago))
print('RMSE Los Angeles: {:,.2f}.'.format(rmse_la))

In [ ]:
print('RMSE ratio between Los Angeles & Chicago: {:.3f}'.format(rmse_la / rmse_chicago))

#### Improving model fit: increase the complexity of the model

A more complex model can find more nuanced patterns in our data. Model complexity can be increased through
- more variables
- transformed variables
- interactions between variables

Since the relationship between price and age are clearly not linear, we can add non-linear terms to our model, for instance the square of age, thereby creating a so-called `polinomial model`. 

<center>
    $price_i = \beta_0 + \beta_1age_i + \beta_2age_i^2 + \epsilon_i$
</center>

In [ ]:
polinomial_model = smf.ols("price ~ age + agesq", data=df_chicago)
polinomial_model_fitted = polinomial_model.fit(cov_type= 'HC1')
polinom_fitted_values_chicago = polinomial_model_fitted.get_robustcov_results().fittedvalues
polinom_fitted_values_la = polinomial_model_fitted.predict(df_la)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x = df_la.age, y = df_la.price, marker = '.')
ax.plot(df_la.age, polinom_fitted_values_la,   color = 'k', label = 'regression line')
ax.hlines( 0, 0, df_la.age.max(), color = 'k', linewidth = 0.5)
plt.title('Original and polinomial fitted values as a function of age - Los Angeles')
plt.legend();

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (12,6))
axs[0].scatter(x = df_chicago.price, y = polinom_fitted_values_chicago, marker = '.', color = 'black')
axs[0].axline([0, 0], [1, 1], color = 'k')
axs[0].set_title('Chicago')
axs[1].scatter(x = df_la.price, y = polinom_fitted_values_la, marker = '.', color = 'k')
axs[1].axline([0, 0], [1, 1], color = 'k')
axs[1].set_title('Los Angeles')
fig.suptitle('Original vs predicted values - polinomial model')

for ax in axs.flat:
    ax.set(xlabel='original', ylabel='fitted/predicted')

for ax in axs.flat:
    ax.label_outer()

In [ ]:
polinomial_rmse_chicago = rmse(df_chicago.price, polinom_fitted_values_chicago)
polinomial_rmse_la = rmse(df_la.price, polinom_fitted_values_la)

print('RMSE Chicago: {:,.2f}.'.format(polinomial_rmse_chicago))
print('RMSE Los Angeles: {:,.2f}.'.format(polinomial_rmse_la))

In [ ]:
print('RMSE ratio between Los Angeles & Chicago: {:.3f}'.format(polinomial_rmse_la / polinomial_rmse_chicago))

<br>
While prediction error (RMSE) improved, it improved more for Chicago - where the model was trained. Perdiction error improved more modestly for the unseen dataset.
<br>
<br>

When we increase model complexity, we get a better fit, but sometimes only for the training data. We call it **overfitting**: our model will be superior for the training dataset but may be completely misleading for another *test* dataset.

`scikit-learn` is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection, model evaluation, and many other utilities.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

#### A super complex model with all possible variations and combinations

In [ ]:
super_model = smf.ols(
    "price ~ age + agesq + odometer + dealer + odometersq + LE*age + XLE*age + SE*age + cond_likenew*age + cond_excellent*age + cond_good*age + cylind6*age + odometer*age + dealer*age", 
    data = df_chicago)
super_model_fitted = super_model.fit(cov_type= 'HC1')
super_fitted_values_chicago = super_model_fitted.get_robustcov_results().fittedvalues
super_fitted_values_la = super_model_fitted.predict(df_la)

In [ ]:
super_rmse_chicago = rmse(df_chicago.price, super_fitted_values_chicago)
super_rmse_la = rmse(df_la.price, super_fitted_values_la)
print('RMSE Chicago: {:,.2f}.'.format(super_rmse_chicago))
print('RMSE Los Angeles: {:,.2f}.'.format(super_rmse_la))

In [ ]:
print('RMSE ratio between Los Angeles & Chicago: {:.3f}'.format(super_rmse_la / super_rmse_chicago))

```text
RMSE has improved meaningfully for Chicago but only slightly for Los Angeles (the unseen data). We are overfitting our model on the training data by using and overly complex model. 
```

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (12,6))
axs[0].scatter(x = df_chicago.price, y = super_fitted_values_chicago, marker = '.', color = 'black')
axs[0].axline([0, 0], [1, 1], color = 'k')
axs[0].set_title('Chicago')
axs[1].scatter(x = df_la.price, y = super_fitted_values_la, marker = '.', color = 'k')
axs[1].axline([0, 0], [1, 1], color = 'k')
axs[1].set_title('Los Angeles')
fig.suptitle('Original vs predicted values')

for ax in axs.flat:
    ax.set(xlabel='original', ylabel='fitted/predicted')

for ax in axs.flat:
    ax.label_outer()

As a reminder:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (12,6))
axs[0].scatter(x = df_chicago.price, y = polinom_fitted_values_chicago, marker = '.', color = 'black')
axs[0].axline([0, 0], [1, 1], color = 'k')
axs[0].set_title('Chicago')
axs[1].scatter(x = df_la.price, y = polinom_fitted_values_la, marker = '.', color = 'k')
axs[1].axline([0, 0], [1, 1], color = 'k')
axs[1].set_title('Los Angeles')
fig.suptitle('Original vs predicted values - polinomial model')

for ax in axs.flat:
    ax.set(xlabel='original', ylabel='fitted/predicted')

for ax in axs.flat:
    ax.label_outer()